<a href="https://colab.research.google.com/github/GabrielRocha97/GabrielRocha97.github.io/blob/main/chartlalaboratory_reading_csv_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
base_path = "/content/drive/MyDrive/Fiverr/chartlaboratory"

In [3]:
import os

os.listdir(base_path)

['chartlalaboratory_reading_csv_files.ipynb',
 'USA Intraday copy_2026-01-03.csv',
 'USA Intraday copy_2026-01-06.csv',
 'USA Intraday copy_2026-01-07-4.csv',
 'USA Intraday copy_2026-01-08.csv',
 'USA Intraday copy_2026-01-09.csv']

In [4]:
import pandas as pd

files = [
    os.path.join(base_path, f)
    for f in os.listdir(base_path)
    if f.endswith(".csv")
]

dfs = [pd.read_csv(f) for f in files]

df = pd.concat(dfs, ignore_index=True)

df.head()

,Symbol,Description,Change from Open % 1 day,Price,Price - Currency,Pre-market High,Pre-market High - Currency,Relative Volume 1 day,Pre-market Change %,Pre-market Gap %,...,Country or region of registration,Industry,Upcoming earnings date,Index,Free float,"Stochastic RSI (3,3,14,14) 30 minutes, K",Williams Percent Range (14) 30 minutes.1,Total common shares outstanding,Simple Moving Average (20) 15 minutes,Relative Volume 1 day.1
0,LVRO,Lavoro Limited,71.594997,1.0016,USD,0.498,USD,348.264455,8.888889,8.888889,...,Brazil,Wholesale distributors,2026-01-28,"NASDAQ Composite, NASDAQ Industrials",6.315927e+06,86.314243,-10.096788,114595436.0,0.462607,NaN
1,BNAI,Brand Engagement Network Inc.,26.824000,3.1706,USD,2.510,USD,0.353272,7.758621,3.448276,...,United States,Packaged software,2026-03-26,"NASDAQ Composite, NASDAQ Computer",2.695967e+06,41.486106,-15.695512,4488080.0,2.440035,NaN
2,SMX,SMX (Security Matters) Public Limited Company,16.410178,22.8746,USD,23.700,USD,5.698523,19.327217,0.366972,...,Ireland,Packaged software,NaN,"NASDAQ Composite, NASDAQ Computer",1.018216e+06,41.337066,-73.757043,1050572.0,22.178120,NaN
3,IRWD,"Ironwood Pharmaceuticals, Inc.",13.381937,5.2099,USD,4.840,USD,18.142470,36.498516,0.000000,...,United States,Pharmaceuticals: major,2026-02-12,"NASDAQ Composite, Russell 2000, Russell 3000, ...",1.566898e+08,54.592181,-22.887550,162679000.0,3.558965,NaN
4,TE,T1 Energy Inc.,13.366690,7.9300,USD,7.040,USD,0.313823,4.640719,2.694611,...,United States,Electrical products,2026-02-26,"Russell 2000, Nasdaq US Small Cap Growth, Mini...",1.883631e+08,40.984678,-1.677852,244891000.0,6.909335,NaN


In [6]:
len(files), files

(5,
 ['/content/drive/MyDrive/Fiverr/chartlaboratory/USA Intraday copy_2026-01-03.csv',
  '/content/drive/MyDrive/Fiverr/chartlaboratory/USA Intraday copy_2026-01-06.csv',
  '/content/drive/MyDrive/Fiverr/chartlaboratory/USA Intraday copy_2026-01-07-4.csv',
  '/content/drive/MyDrive/Fiverr/chartlaboratory/USA Intraday copy_2026-01-08.csv',
  '/content/drive/MyDrive/Fiverr/chartlaboratory/USA Intraday copy_2026-01-09.csv'])

In [8]:
import os
import re
import pandas as pd

dfs = []

for f in files:
    temp = pd.read_csv(f)

    filename = os.path.basename(f)

    # extrai data no formato YYYY-MM-DD
    match = re.search(r"\d{4}-\d{2}-\d{2}", filename)

    if not match:
        raise ValueError(f"Data não encontrada no nome do arquivo: {filename}")

    temp["snapshot_date"] = pd.to_datetime(match.group())

    dfs.append(temp)

df = pd.concat(dfs, ignore_index=True)

In [9]:
df["snapshot_date"].value_counts().sort_index()

,count
snapshot_date,
2026-01-03,157
2026-01-06,175
2026-01-07,1649
2026-01-08,269
2026-01-09,269


In [10]:
df[df["Symbol"] == df["Symbol"].iloc[0]][["Symbol", "snapshot_date", "Price"]]

,Symbol,snapshot_date,Price
0,LVRO,2026-01-03,1.0016


In [11]:
df = df.sort_values(["Symbol", "snapshot_date"])

price_window = (
    df.groupby("Symbol")
      .agg(
          first_price=("Price", "first"),
          last_price=("Price", "last")
      )
      .reset_index()
)

price_window.head()

,Symbol,first_price,last_price
0,AA,54.753,58.8350
1,AAL,15.665,15.9750
2,AAOI,37.920,34.5600
3,AAP,41.360,41.3600
4,ABAT,3.465,4.4273


In [13]:
price_window["pct_price_change"] = (
    (price_window["last_price"] - price_window["first_price"])
    / price_window["first_price"]
) * 100

price_window.head()

,Symbol,first_price,last_price,pct_price_change
0,AA,54.753,58.8350,7.455299
1,AAL,15.665,15.9750,1.978934
2,AAOI,37.920,34.5600,-8.860759
3,AAP,41.360,41.3600,0.000000
4,ABAT,3.465,4.4273,27.772006


In [14]:
ranked_symbols = price_window.sort_values(
    by="pct_price_change",
    ascending=False
)
ranked_symbols.head()

,Symbol,first_price,last_price,pct_price_change
399,CRML,7.7900,14.785,89.794608
835,JCSE,1.2700,2.330,83.464567
1292,RGC,23.8300,38.950,63.449433
1545,UAVS,1.0404,1.690,62.437524
818,IPSC,1.0800,1.725,59.722222


In [15]:
top_10_gainers = ranked_symbols.head(10)

top_10_gainers

,Symbol,first_price,last_price,pct_price_change
399,CRML,7.79000,14.7850,89.794608
835,JCSE,1.27000,2.3300,83.464567
1292,RGC,23.83000,38.9500,63.449433
1545,UAVS,1.04040,1.6900,62.437524
818,IPSC,1.08000,1.7250,59.722222
523,ELVN,15.14000,24.0100,58.586526
674,GLUE,16.01000,25.2100,57.464085
1686,ZNTL,1.78000,2.7443,54.174157
1553,UFG,0.88632,1.3548,52.856756
1428,SRFM,2.18440,3.2944,50.814869


In [16]:
final_table = (
    df.sort_values(["Symbol", "snapshot_date"])
      .groupby("Symbol")
      .agg(
          start_date=("snapshot_date", "first"),
          start_price=("Price", "first"),
          end_date=("snapshot_date", "last"),
          end_price=("Price", "last")
      )
      .reset_index()
)

In [17]:
final_table["pct_change"] = (
    (final_table["end_price"] - final_table["start_price"])
    / final_table["start_price"]
) * 100

In [18]:
final_table = final_table.rename(columns={
    "Symbol": "symbol"
})

In [19]:
final_table = final_table.sort_values(
    by="pct_change",
    ascending=False
)

In [20]:
final_table.head(10)

,symbol,start_date,start_price,end_date,end_price,pct_change
399,CRML,2026-01-03,7.79000,2026-01-09,14.7850,89.794608
835,JCSE,2026-01-06,1.27000,2026-01-07,2.3300,83.464567
1292,RGC,2026-01-06,23.83000,2026-01-08,38.9500,63.449433
1545,UAVS,2026-01-03,1.04040,2026-01-09,1.6900,62.437524
818,IPSC,2026-01-08,1.08000,2026-01-09,1.7250,59.722222
523,ELVN,2026-01-07,15.14000,2026-01-09,24.0100,58.586526
674,GLUE,2026-01-07,16.01000,2026-01-09,25.2100,57.464085
1686,ZNTL,2026-01-07,1.78000,2026-01-09,2.7443,54.174157
1553,UFG,2026-01-06,0.88632,2026-01-08,1.3548,52.856756
1428,SRFM,2026-01-03,2.18440,2026-01-09,3.2944,50.814869
